# Axis Tuning

In [1]:
# Import matplotlib and put it in interactive mode.
%matplotlib notebook
import matplotlib.pyplot as plt
plt.ion()

import math
import numpy as np
import time

from bluesky import RunEngine
from bluesky.callbacks.best_effort import BestEffortCallback
from ophyd import EpicsMotor, EpicsSignal

from apstools.synApps import UserCalcsDevice, setup_lorentzian_swait


IOC = "gp:"

RE = RunEngine({})
RE.subscribe(BestEffortCallback())


def setRandomPeak(calc, motor):
    setup_lorentzian_swait(
        calc, 
        motor, 
        center = -1.5 + 0.5*np.random.uniform(), 
        noise  = 0.2  + 0.2*np.random.uniform(), 
        width  = 0.001 + 0.05*np.random.uniform(), 
        scale  = 1e5*(0.95 + 0.05*np.random.uniform()
        ),
    )

In [2]:
from apstools.devices import AxisTunerMixin
from apstools.plans import TuneAxis


class TunableEpicsMotor(EpicsMotor, AxisTunerMixin):
    pass


m1 = TunableEpicsMotor(f"{IOC}m1", name="m1")
calcs = UserCalcsDevice(IOC, name="calcs")
time.sleep(1)

In [3]:
# change soft motor resolution from 200 steps/rev to 8000 (steps of 0.00025)
_srev = EpicsSignal(m1.prefix+".SREV", name="_srev")
_srev.put(8000)

In [4]:
noisy_calc = calcs.calc1

setRandomPeak(noisy_calc, m1.user_readback)
sigma = calcs.calc1.channels.C.input_value.get()
fwhm = 2*math.sqrt(2* (sigma**2) * math.log(2))

print(f"programmed peak signal: {noisy_calc.channels.D.input_value.get()}")
print(f"programmed center: {noisy_calc.channels.B.input_value.get()}")
print(f"programmed sigma: {sigma}".format(sigma))
print(f"programmed FWHM: {fwhm}".format(fwhm))
print(f"programmed noise: {noisy_calc.channels.E.input_value.get()}")

programmed peak signal: 95511.87579355775
programmed center: -1.2698758825100547
programmed sigma: 0.04619832956187119
programmed FWHM: 0.10878875249924015
programmed noise: 0.342854180129825


In [5]:
noisy = EpicsSignal(noisy_calc.prefix, name="noisy")

In [6]:
m1.tuner = TuneAxis([noisy], m1, signal_name=noisy.name)
# m1.tuner = MyTuneAxis([det], m1, signal_name="det")
m1.tuner.width = 5
m1.tuner.num = 21

m1.move(0)

MoveStatus(done=True, pos=m1, elapsed=0.0, success=True, settle_time=0.0)

In [7]:
RE(m1.tune())



Transient Scan ID: 1     Time: 2022-01-20 16:10:18
Persistent Unique Scan ID: 'fd015aa6-844b-4451-992b-0036cae4038b'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |      noisy |
+-----------+------------+------------+------------+
|         1 | 16:10:20.9 |   -2.50000 |  163.76164 |
|         2 | 16:10:21.3 |   -2.25000 |  231.71770 |
|         3 | 16:10:21.8 |   -2.00000 |  442.83886 |
|         4 | 16:10:22.3 |   -1.75000 |  965.91401 |
|         5 | 16:10:22.8 |   -1.50000 | 4398.23590 |
|         6 | 16:10:23.3 |   -1.25000 | 89646.98539 |
|         7 | 16:10:23.8 |   -1.00000 | 2856.95367 |
|         8 | 16:10:24.3 |   -0.75000 |  896.89729 |
|         9 | 16:10:24.8 |   -0.50000 |  378.13888 |
|        10 | 16:10:25.3 |   -0.25000 |  198.27921 |
|        11 | 16:10:25.8 |    0.00000 |  159.23331 |
|        12 | 16:10:26.3 |    0.25000 |   94.84984 |
|        13 | 16:10:26.8 |    0.50000 |   79.71515 |
|        14 | 16:10:27.3 |    0.75000 |   60.36996 |
|        15 | 16:10:27.8 |    1.00000 |   50.72381 |
|        16 | 16:10:28.3 |    1.25000 |   36.

('fd015aa6-844b-4451-992b-0036cae4038b',)

In [8]:
m1.tuner.width /= 10
RE(m1.tune())



Transient Scan ID: 2     Time: 2022-01-20 16:10:35
Persistent Unique Scan ID: '415b43c6-901d-492a-b4bb-9fab0b305bec'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |      noisy |
+-----------+------------+------------+------------+
|         1 | 16:10:35.6 |   -1.50125 | 4544.44388 |
|         2 | 16:10:35.7 |   -1.47625 | 4917.82585 |
|         3 | 16:10:35.9 |   -1.45125 | 6964.91013 |
|         4 | 16:10:36.1 |   -1.42625 | 9163.87158 |
|         5 | 16:10:36.3 |   -1.40125 | 11570.37522 |
|         6 | 16:10:36.5 |   -1.37625 | 15713.96428 |
|         7 | 16:10:36.7 |   -1.35125 | 24230.55024 |
|         8 | 16:10:36.9 |   -1.32625 | 42270.30327 |
|         9 | 16:10:37.1 |   -1.30125 | 76237.69980 |
|        10 | 16:10:37.3 |   -1.27625 | 90878.57770 |
|        11 | 16:10:37.5 |   -1.25125 | 97910.01753 |
|        12 | 16:10:37.7 |   -1.22625 | 53313.66587 |
|        13 | 16:10:37.9 |   -1.20125 | 37705.70723 |
|        14 | 16:10:38.1 |   -1.17625 | 20459.89763 |
|        15 | 16:10:38.3 |   -1.15125 | 13852.25634 |
|        16 | 16:10:38.5 |   -1.126

('415b43c6-901d-492a-b4bb-9fab0b305bec',)

In [9]:
m1.tuner.width /= 10
RE(m1.tune())



Transient Scan ID: 3     Time: 2021-11-30 16:42:11
Persistent Unique Scan ID: 'fa1be2dd-b1ee-4341-960c-a1e53d3bb474'
New stream: 'primary'


<IPython.core.display.Javascript object>

+-----------+------------+------------+------------+
|   seq_num |       time |         m1 |      noisy |
+-----------+------------+------------+------------+
|         1 | 16:42:11.5 |   -1.22450 | 62430.79022 |
|         2 | 16:42:11.5 |   -1.22200 | 70994.60582 |
|         3 | 16:42:11.6 |   -1.21950 | 73380.25555 |
|         4 | 16:42:11.7 |   -1.21700 | 81461.14601 |
|         5 | 16:42:11.8 |   -1.21450 | 73934.54281 |
|         6 | 16:42:11.9 |   -1.21200 | 102732.89526 |
|         7 | 16:42:12.0 |   -1.20950 | 100576.40224 |
|         8 | 16:42:12.1 |   -1.20700 | 94245.53230 |
|         9 | 16:42:12.2 |   -1.20450 | 99874.43062 |
|        10 | 16:42:12.3 |   -1.20200 | 103063.42739 |
|        11 | 16:42:12.4 |   -1.19950 | 112905.14698 |
|        12 | 16:42:12.5 |   -1.19700 | 113378.40490 |
|        13 | 16:42:12.6 |   -1.19450 | 114397.30787 |
|        14 | 16:42:12.7 |   -1.19200 | 99758.48355 |
|        15 | 16:42:12.8 |   -1.18950 | 85278.49907 |
|        16 | 16:42:12.9 

('fa1be2dd-b1ee-4341-960c-a1e53d3bb474',)